# Parcours d'un réseau de routeurs avec le protocole OSPF

Nous avons étudié dans l’année les protocoles de routage RIP (Routing Information Protocol) et OSPF (Open Shortest Path First) utilisés sur Internet.

Un réseau de routeurs peut être représenté par un graphe non-orienté et pondéré : les sommets sont des ordinateurs ou des routeurs et les arêtes sont pondérées par la métrique du protocole OSPF (liée à la vitesse de transmission de la liaison.)


<img src="Reseau_OSPF.png">

# Modélisation en Python d’un réseau de routeurs et étude de chemins (de routes) utilisant le protocole OSPF.


On peut modéliser ce réseau par une matrice d'adjacence (de dimension 6).

$$mat\_adjacence = \begin{pmatrix} 
0 & 0 & 0  & 10 & 3 & 0\\
0 & 0 & 0  & 1 & 4 & 0\\
0 & 0 & 0  & 9 & 0 & 0\\
10 & 1 & 9 & 0 & 6 & 1\\
3 & 4 & 0 & 6 & 0 & 0\\
0 & 0 & 0 & 1 & 0 & 0\\
\end{pmatrix}$$

<img src="matrice_adjacence_commentee.png">

## 

# Génération d'un réseau aléatoire de routeurs 

In [2]:
import matplotlib.pyplot as plt
from graphviz import Graph
import random 
import os

FORMAT_FICHIER_IMAGE = "png"

def genere_reseau_aleatoire(nb_routeurs):
    """
    Génère aléatoirement un réseau de routeurs
    :param nb_retours: le nombre de routeurs
    :return: une matrice d'adjacence représentant le réseau
    A l'intersection de la ligne lig et de la colonne col, un coefficient non nul indique que les routeurs (lig + 1) et
    (col + 1) sont reliés et que leur métrique de transmission est donné par le coefficient de la matrice
    """
    # On génère une matrice carrée de taille nb_routeurs remplie de zéros à l'aide d'une liste en compréhension 
    matrice_adjacence = [[0 for loop in range(nb_routeurs)] for loop in range(nb_routeurs)]
    for no_routeur in range(nb_routeurs):
        # la liste des voisins possibles est constitué de tous les numéros de routeurs de 0 à no_routeur - 1.
        liste_voisins_possibles = [no for no in range(no_routeur)]
        if len(liste_voisins_possibles) > 0:
            # On extrait un échantillon aléatoire de la liste liste_voisins_possibles de taille aléatoire (comprise entre 1 le nombre de voisins possibles)
            voisins_choisis = random.sample(liste_voisins_possibles,random.randint(1,len(liste_voisins_possibles)))
            for no_routeur_voisin in voisins_choisis:
                    matrice_adjacence[no_routeur][no_routeur_voisin] = random.randint(1,10)
                    # le graphe étant non orienté la matrice est symétrique
                    matrice_adjacence[no_routeur_voisin][no_routeur] = matrice_adjacence[no_routeur][no_routeur_voisin]
    return matrice_adjacence

#exemple de génération d'un réseau de routeurs
nb_routeurs = int(input("Nombre de routeurs souhaités : "))
matrice_adjacence = genere_reseau_aleatoire(nb_routeurs)
print()
print("Matrice d'adjacence : ")
for ligne in matrice_adjacence:
    print(ligne)


Nombre de routeurs souhaités : 4

Matrice d'adjacence : 
[0, 4, 4, 8]
[4, 0, 4, 0]
[4, 4, 0, 0]
[8, 0, 0, 0]


# Visualisation du graphe à l'aide de la bibliothèque Python Graphviz

In [3]:
import os
import time
from IPython.display import display, HTML

def affiche_graphe(matrice_adjacence):
    """
    :param matrice_adjacence: matrice d'adjacence représentant le graphe du réseau de routeurs
    :return: None
    """
    graph = Graph(comment='RIP - OSPF')
    # les arêtes du graphe
    for lig in range(len(matrice_adjacence)):
        for col in range(lig):
            if matrice_adjacence[lig][col] != 0:
                nom_routeur1 = "Routeur_" + str(lig + 1)
                nom_routeur2 = "Routeur_" + str(col + 1)
                metrique = str(matrice_adjacence[lig][col])
                # On créé une arête entre les deux sommets (deux routeurs reperés par (lig + 1) et (col + 1). 
                # Le coefficient de la matrice donne la métrique liant les deux routeurs.
                graph.edge(nom_routeur1, nom_routeur2,label=metrique)
    # On génére le fichier DOT ainsi qu'une image au            
    prefixe_nom_fichier = 'reseau_' + str(len(matrice_adjacence)) 
    with open(prefixe_nom_fichier + '.dot','w') as f:
        f.write(graph.source)
    print()
    print("Fichier au format DOT")
    print(graph.source)
    graph.format = FORMAT_FICHIER_IMAGE
    graph.render(prefixe_nom_fichier)
    nom_fichier = prefixe_nom_fichier + "."+ FORMAT_FICHIER_IMAGE
    times = time.time()
    display(HTML("<img src='" + nom_fichier + "?" + str(times) + "'>"))

nb_routeurs = int(input("Nombre de routeurs souhaités : "))
matrice_adjacence = genere_reseau_aleatoire(nb_routeurs)
print()
print("matrice d'adjacence : ")
for ligne in matrice_adjacence:
    print(ligne)
affiche_graphe(matrice_adjacence)    

Nombre de routeurs souhaités : 12

matrice d'adjacence : 
[0, 1, 0, 10, 8, 9, 0, 0, 0, 0, 8, 7]
[1, 0, 5, 2, 0, 10, 0, 0, 0, 0, 0, 0]
[0, 5, 0, 4, 8, 0, 8, 0, 2, 0, 0, 0]
[10, 2, 4, 0, 5, 0, 0, 7, 9, 3, 7, 0]
[8, 0, 8, 5, 0, 6, 10, 0, 2, 0, 0, 0]
[9, 10, 0, 0, 6, 0, 1, 4, 0, 4, 0, 0]
[0, 0, 8, 0, 10, 1, 0, 0, 0, 10, 5, 0]
[0, 0, 0, 7, 0, 4, 0, 0, 5, 0, 0, 0]
[0, 0, 2, 9, 2, 0, 0, 5, 0, 5, 0, 0]
[0, 0, 0, 3, 0, 4, 10, 0, 5, 0, 0, 0]
[8, 0, 0, 7, 0, 0, 5, 0, 0, 0, 0, 0]
[7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Fichier au format DOT
// RIP - OSPF
graph {
	Routeur_2 -- Routeur_1 [label=1]
	Routeur_3 -- Routeur_2 [label=5]
	Routeur_4 -- Routeur_1 [label=10]
	Routeur_4 -- Routeur_2 [label=2]
	Routeur_4 -- Routeur_3 [label=4]
	Routeur_5 -- Routeur_1 [label=8]
	Routeur_5 -- Routeur_3 [label=8]
	Routeur_5 -- Routeur_4 [label=5]
	Routeur_6 -- Routeur_1 [label=9]
	Routeur_6 -- Routeur_2 [label=10]
	Routeur_6 -- Routeur_5 [label=6]
	Routeur_7 -- Routeur_3 [label=8]
	Routeur_7 -- Routeur_5 [label=10]


# Recherche d'un chemin de poids minimal pour acheminer des données entre deux routeurs

## On utilise la bibiothèque ParcoursGraphes développée ensemble en TP dans l'année

Plus particulièrement on va utiliser la fonction <code>Dijkstra(graphe,depart,arrivee)</code> qui implémente l'algorithme de<b>Dijkstra</b> de recherche de chemin de poids minimal dans un graphe pondéré.

## Conversion de la matrice d'adjacence en une liste d'adjacence (avec un dictionnaire Python)

La fonction <code>Dijkstra(graphe,depart,arrivee)</code> attend en entrée un dictionnaire dont les clefs sont les sommets du graphe et les valeurs la liste des voisins d'un sommet donné.

Nous devons donc écrire une fonction Python qui convertit la <b>matrice d'adjacence<b> en une <b>liste d'adjacence</b>. (Nous l'avons déjà fait en début d'année via un exercice sous Repl.It ;) !).

In [3]:
def matrice_adjacence_vers_liste_adjacence(matrice):
    """
    Convertit une matrice d'adjacence représentant un réseau de routeurs en une liste d'adjacence
    dictionnaire dont les clefs sont les noms des routeurs et les valeurs la liste des voisins du routeur donné sous la forme
    d'un tuple : (nom_routeur_voisin, distance)
    :param matrice:
    :return: liste d'adjacence
    """
    liste_adjacence = {}
    for lig in range(len(matrice)):
        for col in range(lig):
            if matrice[lig][col] != 0:
                nom_routeur1 = "Routeur_" + str(lig + 1)
                nom_routeur2 = "Routeur_" + str(col + 1)
                if nom_routeur1 in liste_adjacence.keys():
                    liste_adjacence[nom_routeur1].append((nom_routeur2,matrice[lig][col]))
                else:
                    liste_adjacence[nom_routeur1] = [(nom_routeur2, matrice[lig][col])]
                if nom_routeur2 in liste_adjacence.keys():
                    liste_adjacence[nom_routeur2].append((nom_routeur1,matrice[lig][col]))
                else:
                    liste_adjacence[nom_routeur2] = [(nom_routeur1, matrice[lig][col])]
    return liste_adjacence

nb_routeurs = int(input("Nombre de routeurs souhaités : "))
matrice_adjacence = genere_reseau_aleatoire(nb_routeurs)
print()
print("matrice d'adjacence : ")
for ligne in matrice_adjacence:
    print(ligne)
print()
liste_adjacence = matrice_adjacence_vers_liste_adjacence(matrice_adjacence)
print("Liste adjacence correspondante :")
print(liste_adjacence)



Nombre de routeurs souhaités : 3

matrice d'adjacence : 
[0, 4, 5]
[4, 0, 0]
[5, 0, 0]

Liste adjacence correspondante :
{'Routeur_2': [('Routeur_1', 4)], 'Routeur_1': [('Routeur_2', 4), ('Routeur_3', 5)], 'Routeur_3': [('Routeur_1', 5)]}


## Utilisation de la fonction <code>Dijkstra(graphe,depart,arrivee)</code> de notre bibliothèque de parcours de graphes

Le code suivant recherche un réseau aléatoire de 10 routeurs dont un chemin minimal donné par Dijkstra entre deux sommets composé d'au moins 6 routeurs.

In [4]:
import ParcoursGraphes as pg
from IPython.display import display, HTML
import time

FORMAT_FICHIER_IMAGE = "png"

def affiche_graphe_complet(matrice_adjacence,routeur_depart=None,routeur_arrivee=None,ospf_max=None,rip_max=None):
    """
    :param graphe:
    :return:
    """
    graph = Graph(comment='OSPF')
  
    # Les sommets du graphe
    if routeur_depart:
        no_routeur_depart = int(routeur_depart.split("_")[1])
        no_routeur_arrivee = int(routeur_arrivee.split("_")[1])
        for lig in range(len(matrice_adjacence)):
            nom_routeur = "Routeur_" + str(lig + 1)
            if no_routeur_depart == lig + 1 or no_routeur_arrivee == lig + 1:
                graph.node(nom_routeur,color='green',style='filled')
            elif ospf_max and nom_routeur in ospf_max[1]:
                graph.node(nom_routeur,color='green')
    # les arêtes du graphe
    for lig in range(len(matrice_adjacence)):
        for col in range(lig):

            if matrice_adjacence[lig][col] != 0:
                nom_routeur1 = "Routeur_" + str(lig + 1)
                nom_routeur2 = "Routeur_" + str(col + 1)
                metrique = str(matrice_adjacence[lig][col])
                if ospf_max and nom_routeur1 in ospf_max[1] and nom_routeur2 in ospf_max[1] and abs(ospf_max[1].index(nom_routeur1) - ospf_max[1].index(nom_routeur2))==1:
                    graph.edge(nom_routeur1, nom_routeur2, label=metrique,color='green',size='5')
                    if rip_max and nom_routeur1 in rip_max[1] and nom_routeur2 in rip_max[1] and abs(rip_max[1].index(nom_routeur1) - rip_max[1].index(nom_routeur2)) == 1:
                        graph.edge(nom_routeur2, nom_routeur1, label=metrique, color='red', size='5')
                elif rip_max and nom_routeur1 in rip_max[1] and nom_routeur2 in rip_max[1] and abs(rip_max[1].index(nom_routeur1) - rip_max[1].index(nom_routeur2))==1:
                    graph.edge(nom_routeur1, nom_routeur2, label=metrique,color='red',size='5')
                else:
                    graph.edge(nom_routeur1, nom_routeur2,label=metrique)
    prefixe_nom_fichier = 'reseau_' + str(len(matrice_adjacence)) 
    with open(prefixe_nom_fichier + '.dot','w') as f:
        f.write(graph.source)
    graph.format = FORMAT_FICHIER_IMAGE
    graph.render(prefixe_nom_fichier)
    nom_fichier = prefixe_nom_fichier + "."+ FORMAT_FICHIER_IMAGE
    times = time.time()
    display(HTML("<img src='" + nom_fichier + "?" + str(times) + "'>"))

def exemple_OSPF(nb_routeurs):
   matrice_adjacence = genere_reseau_aleatoire(nb_routeurs)
   liste_adjacence = matrice_adjacence_vers_liste_adjacence(matrice_adjacence)
   routeur_depart,routeur_arrivee = random.sample(list(liste_adjacence.keys()),2)
   chemin_min = pg.Dijkstra(liste_adjacence,routeur_depart,routeur_arrivee)
   return matrice_adjacence,liste_adjacence,routeur_depart,routeur_arrivee,chemin_min

while True:
   matrice_adjacence,liste_adjacence,routeur_depart,routeur_arrivee,chemin_min = exemple_OSPF(10)
   if len(chemin_min[1]) > 6:
       print("chemin minimal avec OSPF : ",chemin_min)
       affiche_graphe_complet(matrice_adjacence,routeur_depart,routeur_arrivee,chemin_min)
       break

chemin minimal avec OSPF :  (26, ['Routeur_8', 'Routeur_10', 'Routeur_5', 'Routeur_1', 'Routeur_4', 'Routeur_7', 'Routeur_9'])


## Comparaison des chemins avec les protocoles RIP et OSPF

Le protocole RIP ne prend pas en compte les vitesses de transmission entre les routeurs.
Il compte juste le nombre de sauts d'un routeur à un autre.
Ce qui revient à considérer un graphe <b>non pondéré</b>.

Le code suivant recherche un réseau aléatoire de 7 routeurs dont : 
<ul>
    <li>un chemin minimal entre 2 routeurs avec OSPF est composé d'au moins 4 routeurs;</li>
    <li>un chemin minimal avec le protocole RIP entre les 2 mêmes sommets avec de moins de sauts qu'avec OSP mais une route pondérée plus lente.</li>
    </ul>
La route OSPF est indiquée en rouge et la route RIP en vert.

In [17]:
def cherche_chemin_plus_court_RIP(graphe,depart,arrivee):
    """
    @param graphe: 
    @param depart: 
    @param arrivee: 
    @return: 
    """
    graphe_non_pondere = {}
    for routeur in graphe.keys():
        graphe_non_pondere[routeur] = [(voisin[0],1) for voisin in graphe[routeur]]
    chemin_min = pg.Dijkstra(graphe_non_pondere,depart,arrivee)
    poids = 0
    for ind in range(len(chemin_min)-1):
        voisins = graphe[chemin_min[1][ind]]
        for voisin in voisins:
            if voisin[0] == chemin_min[1][ind+1]:
                poids += voisin[1]
    return (poids,chemin_min[1])

def exemple_RIP_et_OSPF(nb_routeurs):
   matrice_adjacence = genere_reseau_aleatoire(nb_routeurs)
   liste_adjacence = matrice_adjacence_vers_liste_adjacence(matrice_adjacence)
   routeur_depart,routeur_arrivee = random.sample(list(liste_adjacence.keys()),2)
   chemin_min_OSPF = pg.Dijkstra(liste_adjacence,routeur_depart,routeur_arrivee)
   chemin_min_RIP = cherche_chemin_plus_court_RIP(liste_adjacence,routeur_depart,routeur_arrivee) 
   return matrice_adjacence,liste_adjacence,routeur_depart,routeur_arrivee,chemin_min_OSPF,chemin_min_RIP

while True:
   matrice_adjacence,liste_adjacence,routeur_depart,routeur_arrivee,chemin_min_OSPF,chemin_min_RIP = exemple_RIP_et_OSPF(7)
   if len(chemin_min_OSPF[1]) > 4 and len(chemin_min_RIP[1]) > 3 and len(chemin_min_OSPF[1]) > len(chemin_min_RIP[1]) - 2 and chemin_min_OSPF[0] < chemin_min_RIP[0]:
       print("chemin minimal avec OSPF : ",chemin_min_OSPF)
       print("chemin minimal avec RIP : ",chemin_min_RIP)
       affiche_graphe_complet(matrice_adjacence,routeur_depart,routeur_arrivee,chemin_min_OSPF,chemin_min_RIP)
       break



chemin minimal avec OSPF :  (8, ['Routeur_6', 'Routeur_7', 'Routeur_5', 'Routeur_1', 'Routeur_3'])
chemin minimal avec RIP :  (9, ['Routeur_6', 'Routeur_2', 'Routeur_1', 'Routeur_3'])
